In [1]:
#!pip install pickle5
#!pip install transformers

#! pip3 install torch==1.5.0 transformers==3.4.0
#! pip install faiss-gpu cudatoolkit=10.0 -c pytorch

In [2]:
import pickle5 as pickle
import pandas as pd
#from drive.MyDrive.CLIR.preprocess_data import create_cleaned_token_embedding, load_embeddings
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.models.predict_model import MAP_score 
from src.cross_lingual_embeddings.text_encoders import TextEncoders
from src.data.utils import get_sentence_pairs


#from google.colab import drive
#drive.mount('/content/drive')
 
#drive.flush_and_unmount()

Mounted at /content/drive


# Preprocess Data

In [19]:
with open('/content/drive/MyDrive/crosslingual-information-retrieval/data/interim/europarl_en_de.pkl', "rb") as fh:
    original_sentence_pairs = pickle.load(fh)

feature_retrieval = pd.read_feather("/content/drive/MyDrive/crosslingual-information-retrieval/data/processed/feature_retrieval_en_de.feather")
feature_retrieval = feature_retrieval.rename(columns={"id_source": "source_id", "id_target": "target_id"})
dataset_pairs = get_sentence_pairs(original_sentence_pairs, feature_retrieval)

# Unsupervised: XLM-R First Layer

In [20]:
# Text Encoder First Layer
print("--------------------------------")
print("\nCreate  Text Encoder First Layer model")


xlm_r = TextEncoders("xlm-r")
source_known = {}
source_embedding = []
for sentence in tqdm(dataset_pairs["text_source"].values.tolist()):
    if sentence in source_known.keys():
        source_embedding.append(source_known[sentence])
        continue
    sentence_embedding = xlm_r.create_embedding_for_each_term(sentence, use_layer=1)
    sentence_embedding = np.average(sentence_embedding[0], axis=0)
    source_known[sentence] = sentence_embedding
    source_embedding.append(sentence_embedding)

target_known = {}
target_embedding = []
for sentence in tqdm(dataset_pairs["text_target"].values.tolist()):
    if sentence in target_known.keys():
        target_embedding.append(target_known[sentence])
        continue
    sentence_embedding = xlm_r.create_embedding_for_each_term(sentence, use_layer=1)
    sentence_embedding = np.average(sentence_embedding[0], axis=0)
    target_known[sentence] = sentence_embedding
    target_embedding.append(sentence_embedding)

cosine_similarity_total = []
for i in tqdm(range(len(target_embedding))):
    cos = cosine_similarity(source_embedding[i].reshape(1, -1), target_embedding[i].reshape(1, -1), dense_output=True)[0][0]
    cosine_similarity_total.append(cos)


cosine_similarity_total = [[1-i, i] for i in cosine_similarity_total]
print("The MAP score on test set: {:.4f}".format(MAP_score(feature_retrieval['source_id'],feature_retrieval['Translation'],cosine_similarity_total)))

--------------------------------

Create  Text Encoder First Layer model


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [23]:
cosine_similarity_total = []
for i in tqdm(range(len(target_embedding))):
    cos = cosine_similarity(source_embedding[i].reshape(1, -1), target_embedding[i].reshape(1, -1), dense_output=True)[0][0]
    cosine_similarity_total.append(cos)


cosine_similarity_total = [[1-i, i] for i in cosine_similarity_total]
print("The MAP score on test set: {:.4f}".format(MAP_score(feature_retrieval['source_id'],feature_retrieval['Translation'],cosine_similarity_total)))

The MAP score on test set: 0.0355


# Unsupervised: XLM-R Last Layer

In [24]:
source_known = {}
source_embedding = []
for sentence in tqdm(dataset_pairs["text_source"].values.tolist()):
    if sentence in source_known.keys():
        source_embedding.append(source_known[sentence])
        continue
    sentence_embedding = xlm_r.create_embedding_for_each_term(sentence, use_layer=1)
    sentence_embedding = np.average(sentence_embedding[0], axis=0)
    source_known[sentence] = sentence_embedding
    source_embedding.append(sentence_embedding)




target_known = {}
target_embedding = []
for sentence in tqdm(dataset_pairs["text_target"].values.tolist()):
    if sentence in target_known.keys():
        target_embedding.append(target_known[sentence])
        continue
    sentence_embedding = xlm_r.create_embedding_for_each_term(sentence, use_layer=12)
    sentence_embedding = np.average(sentence_embedding[0], axis=0)
    target_known[sentence] = sentence_embedding
    target_embedding.append(sentence_embedding)

cosine_similarity_total = []
for i in tqdm(range(len(target_embedding))):
    cos = cosine_similarity(source_embedding[i].reshape(1, -1), target_embedding[i].reshape(1, -1), dense_output=True)[0][0]
    cosine_similarity_total.append(cos)

cosine_similarity_total = [[1-i, i] for i in cosine_similarity_total]
print("The MAP score on test set: {:.4f}".format(MAP_score(feature_retrieval['source_id'],feature_retrieval['Translation'],cosine_similarity_total)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



The MAP score on test set: 0.0060
